In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ninja
!pip install wandb

In [12]:
import wandb

In [ ]:
wandb.init(project="Self-Correction-Human-Parsing-For_Binary-Images")

In [14]:
config = wandb.config
config.learning_rate = 0.001

In [15]:
%cd /content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing

/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing


In [ ]:
!mkdir train_image
!mkdir train_segmantation
!mkdir val_images
!mkdir val_segmantation

In [16]:
!pip install -r requirements.txt

In [17]:
import gdown

In [ ]:
url = "https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE"
out = "weights/exp-schp-201908270938-pascal-person-part.pth"
gdown.download(url,out)

In [ ]:
!python simple_extractor.py --dataset 'pascal' --model-restore "weights/exp-schp-201908270938-pascal-person-part.pth" --input-dir "./input/JPEGImages" --output-dir "./output"

In [ ]:
import os
import cv2

path = "/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/output"
list_files = os.listdir(path)[0:1000]
f1 =  open('/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/train_id.txt', 'w')
f2 = open('/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/val_id.txt', 'w')

for i, im in enumerate(list_files):
  img = cv2.imread(os.path.join(path, im))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray[img_gray > 0] = 255
  img_gray[img_gray == 0] = 0       
  # print(im)
  _, tail = os.path.split(im)
  # # print(tail)
  name = tail.split('.')[0]
  name = name.split('_')[1]
  # print(name)

  if i < 800:
    cv2.imwrite(f"/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/train_images/{name}.jpg", img_gray)
    f1.write(f"{name}")
    f1.write('\n')

  else:
    cv2.imwrite(f"/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/val_images/{name}.jpg", img_gray)
    f2.write(f"{name}")
    f2.write('\n')

  
f1.close()
f2.close()

In [ ]:
# Create labels

import os
import cv2
import numpy as np

path = "/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/output"
list_files = os.listdir(path)[0:1000]


for i, im in enumerate(list_files):
  img_rgb = cv2.imread(os.path.join(path, im))
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
  # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_gray[img_gray == 0] = 0
  img_gray[img_gray == 15] = 1
  img_gray[img_gray == 38] = 2
  img_gray[img_gray == 53] = 3
  img_gray[img_gray == 75] = 4
  img_gray[img_gray == 90] = 5
  img_gray[img_gray == 113] = 6
  # print(im)
  _, tail = os.path.split(im)
  name = tail.split('.')[0]
  name = name.split('_')[1]
  
  

  if i < 800:
    cv2.imwrite(f"/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/train_segmentations/{name}.png", img_gray)
  else:
    cv2.imwrite(f"/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/val_segmentations/{name}.png", img_gray)

In [ ]:
# visualization

import matplotlib.pyplot as plt
import cv2

fig = plt.figure(figsize=(10, 10))

img = cv2.imread("/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/input/JPEGImages/2500_91.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.subplot(141) 
plt.imshow(img)

img1 = cv2.imread("/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/output/2500_91.png")
plt.subplot(142) 
plt.imshow(img1)

img = cv2.imread("/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/val_images/91.jpg")
plt.subplot(143) 
plt.imshow(img, cmap='gray')

img = cv2.imread("/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/dataset/val_segmentations/91.png")
plt.subplot(144) 
plt.imshow(img, cmap='gray')

In [9]:
%cd /content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing
!python train.py --data-dir dataset --num-classes 7 --batch-size 3 --imagenet-pretrain ./pretrain_model/resnet101-imagenet.pth

/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing
wandb: Currently logged in as: cyber (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.12.11
wandb: Run data is saved locally in /content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing/wandb/run-20220317_070944-37tdbpwf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run glowing-forest-25
wandb: ⭐️ View project at https://wandb.ai/cyber/Face_Detection
wandb: 🚀 View run at https://wandb.ai/cyber/Face_Detection/runs/37tdbpwf
Namespace(arch='resnet101', batch_size=3, cycle_epochs=10, data_dir='dataset', epochs=20, eval_epochs=10, gpu='0,1,2', ignore_label=255, imagenet_pretrain='./pretrain_model/resnet101-imagenet.pth', input_size='473,473', lambda_c=0.1, lambda_e=1, lambda_s=1, learning_rate=0.001, log_dir='./log', model_restore='./log/checkpoint.pth.tar', momentum=0.9, num_classes=7, random_mirror=False, random_scale=False, schp_restore='./log/schp_checkpoin

In [ ]:
%cd /content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing
!python simple_extractor.py --dataset 'pascal' --model-restore "./log/checkpoint_20.pth.tar" --input-dir './dataset/train_images' --output-dir './binary_output'

/content/drive/MyDrive/Human_parsing/Self-Correction-Human-Parsing
Evaluating total class number 7 with ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']
 98% 775/787 [05:08<00:04,  2.45it/s]